In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
s = requests.Session()
response = s.get(url, timeout=10)
response

<Response [200]>

In [4]:
# parse response content to html
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
#title of wikipedia page
soup.title.string

'List of postal codes of Canada: M - Wikipedia'

In [6]:
#get right table to scrap
right_table = soup.find('table',{"class":'wikitable sortable'})

In [7]:
# Number of columns in the table
for row in right_table.findAll("tr"):
    cells = row.findAll('td')

len(cells)

3

In [8]:
# number of rows in the table including header
rows = right_table.findAll("tr")
len(rows)

181

In [9]:
# header attributes of the table
header = [th.text.rstrip() for th in rows[0].find_all('th')]
print(header)
print('------------')
print(len(header))

['Postal Code', 'Borough', 'Neighbourhood']
------------
3


In [10]:
lst_data = []
for row in rows[1:]:
            data = [d.text.rstrip() for d in row.find_all('td')]
            lst_data.append(data)

In [11]:
# select also works as find_all
lst_data1 = []
for row in rows[1:]:
            data = [d.text.rstrip() for d in row.select('td')]
            lst_data1.append(data)

In [12]:
# sample records
lst_data1[0:3]

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods']]

In [13]:
#Scrap the data and append to respective lists

c1=[]
c2=[]
c3=[]
for row in right_table.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        c1.append(cells[0].find(text=True))
        c2.append(cells[1].find(text=True)) 
        c3.append(cells[2].find(text=True))



In [14]:
# create a dictionary
#d = dict([(x,0) for x in header])
d = dict([('PostalCode', 0), ('Borough', 0), ('Neighbourhood', 0)])
d

{'PostalCode': 0, 'Borough': 0, 'Neighbourhood': 0}

In [15]:
# append dictionary with corresponding data list.
d['PostalCode'] = c1
d['Borough']= c2
d['Neighbourhood']=c3

In [24]:
# convert dict to DataFrame
df = pd.DataFrame(d)

# Top 5 records
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [25]:
df = df.drop(df[df.Borough == 'Not assigned\n'].index)
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
5,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
6,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"


In [26]:
df = df.replace('\n',' ', regex=True)
#df.rename(columns={'Postal Code': 'PostalCode', 'Borough': 'Boroug', 'Neighbourhood': 'Neighbourhoo'})
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [22]:
df.tail()

,PostalCode,Borough,Neighbourhood
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [34]:
#Combine neighborhoods that exist in one postal code area
df.groupby('PostalCode')['Neighbourhood'].apply(' '.join).reset_index(drop=True)
df = df.reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [40]:
df['Neighbourhood'][1]

'Parkwoods '

In [43]:
#Replace "Not assigned" neighbourhoods for the respective Borough value
i=0
for index, row in df.iterrows():
    if df.Neighbourhood[index] == 'Not assigned':
        i += 1
        df.Neighbourhood[index] = df.Borough[index]
        
if i==0:
    print('No rows with "Not assigned" neighbourhoods')
else:
    print(i,' rows with "Not assgined" neighbourhoods replaced')
        

No rows with "Not assigned" neighbourhoods


In [45]:
df.shape

(103, 3)